In [0]:
# Install the PyDrive wrapper & import libraries.
# This only needs to be done once per notebook.
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials


import tensorflow as tf
from tensorflow.python.keras.models import Model
from tensorflow.python.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.python.keras.applications import xception
from google.colab import files

In [0]:
# Authenticate and create the PyDrive client.
# This only needs to be done once per notebook.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)


# Download a file based on its file ID.
#
files = ['10bcf4_bV-B_GmX_40BDjD7BZjRve-ciw',
'1gdTW4bRF9reGffYwgQoKSfqMjjXoWKNq',
'1-4VuiQHr0mCu8IK28ACPYDxpouxKUNvo',
'19meBzmNDy570ch88u812KuEFXsyt4yPc',
'1Z8bY6e-LWdVd6y-TP3qEe75AURytvGxY',
'1-PpJOwSCbpleYD0Ss81Sm6k-mQvpQx7L',
'1PsUeYCptA1RTpSvQAk7vidPIy5qX0KTN',
'1Qvl-jozmmU2YqOt395Beuuph4ivYJb0T',
'1QZjABkrELGxL7fwYXXD0gKammp86bhhn',
'1ePsQIhQUMPFC7VdFzOhIY0Uf5fZO2igU',
'1RniG4-dOuyBCmTRjfSCH4XWblGs4_wLW',
'1_MqKcZQqzvIWTo3A1y5V54G2ewcDhbEY']
for i in range(12):
    file_id = files[0]
    downloaded = drive.CreateFile({'id': file_id})
    downloaded.GetContentFile('train-' + str(i))

In [3]:
base_model = xception.Xception(include_top=False, weights='imagenet', input_shape=(299, 299,3))
x = base_model.output

x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)


predictions = Dense(196, activation='softmax')(x)

# this is the model we will train
model = Model(inputs=base_model.input, outputs=predictions)

# for layer in base_model.layers:
#     layer.trainable = False

# compile the model (should be done *after* setting layers to non-trainable)
run_config = tf.estimator.RunConfig()
run_config = run_config.replace(keep_checkpoint_max=25, save_summary_steps=10)
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
est = tf.keras.estimator.model_to_estimator(model, model_dir='out', config=run_config)

INFO:tensorflow:Using the Keras model from memory.
INFO:tensorflow:Using config: {'_model_dir': 'out', '_tf_random_seed': None, '_save_summary_steps': 10, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': None, '_keep_checkpoint_max': 25, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7fb2c9c5b780>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


In [0]:
DATASET_FILE = 'cars_annos'
seed = 123
n_threads = 2
image_size=299
BATCH_SIZE=32

In [0]:
def parse_example_proto(example_serialized):
    # Dense features in Example proto.
    feature_map = {
        'image/encoded': tf.FixedLenFeature([], dtype=tf.string,
                                            default_value=''),
        'image/class/label': tf.FixedLenFeature([1], dtype=tf.int64,
                                                default_value=-1),
        'image/class/text': tf.FixedLenFeature([], dtype=tf.string,
                                               default_value=''),
        'image/height': tf.FixedLenFeature([1], dtype=tf.int64,
                                                default_value=-1),
        'image/width': tf.FixedLenFeature([1], dtype=tf.int64,
                                                default_value=-1),
    }
    sparse_float32 = tf.VarLenFeature(dtype=tf.float32)
    # Sparse features in Example proto.
    feature_map.update(
        {k: sparse_float32 for k in ['image/object/bbox/xmin',
                                     'image/object/bbox/ymin',
                                     'image/object/bbox/xmax',
                                     'image/object/bbox/ymax']})

    features = tf.parse_single_example(example_serialized, feature_map)
    label = tf.cast(features['image/class/label'], dtype=tf.int32)
    label = tf.subtract(label, 1)
    xmin = tf.cast(tf.reshape(features['image/object/bbox/xmin'].values, shape=[]), dtype=tf.int32)
    ymin = tf.cast(tf.reshape(features['image/object/bbox/ymin'].values, shape=[]), dtype=tf.int32)
    xmax = tf.cast(tf.reshape(features['image/object/bbox/xmax'].values, shape=[]), dtype=tf.int32)
    ymax = tf.cast(tf.reshape(features['image/object/bbox/ymax'].values, shape=[]), dtype=tf.int32)
    
    tg_y = tf.subtract(ymax, ymin)
    tg_x = tf.subtract(xmax, xmin)

    return features['image/encoded'], label, xmin, ymin, tg_x, tg_y, features['image/class/text']

In [0]:
def decode_jpeg(image_buffer, scope=None):
    """Decode a JPEG string into one 3-D float image Tensor.

    Args:
      image_buffer: scalar string Tensor.
      scope: Optional scope for name_scope.
    Returns:
      3-D float Tensor with values ranging from [0, 1).
    """
    with tf.name_scope(values=[image_buffer], name=scope,
                       default_name='decode_jpeg'):
        # Decode the string as an RGB JPEG.
        # Note that the resulting image contains an unknown height and width
        # that is set dynamically by decode_jpeg. In other words, the height
        # and width of image is unknown at compile-time.
        image = tf.image.decode_jpeg(image_buffer, channels=3)

        # After this point, all image pixels reside in [0,1)
        # until the very end, when they're rescaled to (-1, 1).  The various
        # adjust_* ops all require this range for dtype float.
        image = tf.image.convert_image_dtype(image, dtype=tf.float32)
        return image

In [0]:
def apply_with_random_selector(x, func, num_cases):
    """Computes func(x, sel), with sel sampled from [0...num_cases-1].

    Args:
      x: input Tensor.
      func: Python function to apply.
      num_cases: Python int32, number of cases to sample sel from.

    Returns:
      The result of func(x, sel), where func receives the value of the
      selector as a python integer, but sel is sampled dynamically.
    """
    
    sel = tf.random_uniform([], maxval=num_cases, dtype=tf.int32)
    # Pass the real x only to one of the func calls.
    return control_flow_ops.merge([
        func(tf.python.ops.control_flow_ops.switch(x, tf.equal(sel, case))[1], case)
        for case in range(num_cases)])[0]

def image_preprocessing(image_buffer, xmin, ymin, tg_x, tg_y, train=True):
#     if bbox is None:
#         raise ValueError('Please supply a bounding box.')

    image = decode_jpeg(image_buffer)

    if train:
#         distorted_bbox = tf.image.sample_distorted_bounding_box(tf.shape(image), bounding_boxes=bbox, min_object_covered=.8,
#                                                aspect_ratio_range=(0.05, 1 / ratio), area_range=(0.05, 1.0), max_attempts=100)
#         bbox_begin, bbox_size, distort_bbox = distorted_bbox
        
#         distorted_image = tf.slice(image, bbox_begin, bbox_size)
        distorted_image = tf.image.crop_to_bounding_box(image, ymin, xmin, tg_y, tg_x)
#         rnd_val = tf.random_uniform([], maxval=4, dtype=tf.int32)
        distorted_image = tf.image.resize_images(distorted_image, [image_size, image_size],
                                                 method=0)
        distorted_image.set_shape([image_size, image_size, 3])
        distorted_image = tf.image.random_flip_left_right(distorted_image)
        
    else:
#         image = eval_image(image, height, width)
        pass

    return distorted_image


In [0]:
def parse_fn(example):
    image, label, xmin, ymin, tg_x, tg_y, _ = parse_example_proto(example)
    image = image_preprocessing(image, xmin, ymin, tg_x, tg_y)
    return {'input_1':image}, label
    

In [0]:
def input_fn():
    with tf.device('/cpu:0'):
        files = tf.data.Dataset.list_files("./train-*")
        dataset = files.interleave(tf.data.TFRecordDataset, 1)
        dataset = dataset.apply(tf.contrib.data.shuffle_and_repeat(1150))
        dataset = dataset.map(map_func=parse_fn, num_parallel_calls=n_threads)
        dataset = dataset.batch(batch_size=BATCH_SIZE)
        dataset = dataset.prefetch(buffer_size=BATCH_SIZE)
        iterator = dataset.make_one_shot_iterator()
        batch_features, batch_labels = iterator.get_next()

    return batch_features, batch_labels

In [0]:
with tf.device('/gpu:0'):
    est.train(input_fn=input_fn)

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from out/model.ckpt-1964
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 1965 into out/model.ckpt.
INFO:tensorflow:loss = 99442.31, step = 1965
INFO:tensorflow:global_step/sec: 0.4581
INFO:tensorflow:loss = 102060.92, step = 2065 (218.293 sec)
INFO:tensorflow:global_step/sec: 0.46884
INFO:tensorflow:loss = 102384.21, step = 2165 (213.292 sec)
INFO:tensorflow:Saving checkpoints for 2243 into out/model.ckpt.
INFO:tensorflow:global_step/sec: 0.464338
INFO:tensorflow:loss = 100897.11, step = 2265 (215.361 sec)
INFO:tensorflow:global_step/sec: 0.46861
INFO:tensorflow:loss = 127147.81, step = 2365 (213.397 sec)
INFO:tensorflow:global_step/sec: 0.468523
INFO:tensorflow:loss = 110272.34, step = 2465 (213.436 sec)
INFO:tensorflow:Savi

In [21]:
est.latest_checkpoint()

'out/model.ckpt-1964'

In [0]:
!ls